In [61]:
import video_data_classes as vid
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch
from skimage import io
from importlib import reload
from IPython.core.debugger import set_trace
from torch.utils.data import Dataset, DataLoader

In [62]:
#network = SuperResNetwork()
#criterion = nn.MSELoss()
#optimizer = optim.Rprop(network.parameters())

video_directory = r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs'
preprocess = transforms.Compose([vid.LightenImageTransformer(),
                                vid.MinMaxTransformer()])
dataset = vid.VideoDataset(video_directory, 10, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

In [ ]:
for i, data in enumerate(dataloader):
    train_x = torch.stack(data['X']).permute(0, 1, 4, 2, 3)
    train_y = torch.stack(data['Y']).permute(0, 1, 4, 2, 3)
    
    optimizer.zero_grad()
    
    outputs = network(train_x)
    loss = criterion(outputs, train_y)
    loss.backward()
    optimizer.step()

In [63]:
for i, data in enumerate(dataloader):
    set_trace()
    print(i)

> <ipython-input-63-35179904f091>(3)<module>()
      1 for i, data in enumerate(dataloader):
      2     set_trace()
----> 3     print(i)



ipdb>  type(data['X'])


<class 'list'>


ipdb>  type(data['X'][0])


<class 'torch.Tensor'>


...

*** TypeError: transpose() received an invalid combination of arguments - got (int, int, int, int, int), but expected one of:
 * (name dim0, name dim1)
 * (int dim0, int dim1)


ipdb>  torch.stack(data['X']).permute(0, 1, 4, 2, 3).shape


torch.Size([10, 4, 3, 256, 256])


ipdb>  q


BdbQuit: 

In [ ]:
data = vid.VideoDataset(r'C:\Users\John\PythonVenvs\VideoSuperResolution\Scripts\Raw Data\Raw, Half-Size, and PNGs', 10)

In [ ]:
reload(vid)

In [60]:
torch.Tensor([1, 2, 3, 5])

tensor([1., 2., 3., 5.])

In [ ]:
data[5]